In [1]:
class Schedule:
    def __init__(self, num_subjects, num_rooms, num_days, num_sections, subject_members, room_capacities, subject_pairs):
        self.num_subjects = num_subjects
        self.num_rooms = num_rooms
        self.num_days = num_days
        self.num_sections = num_sections
        self.subject_members = subject_members
        self.room_capacities = room_capacities
        self.subject_pairs = subject_pairs
        self.schedules = [[[None for _ in range(num_sections)] for _ in range(num_days)] for _ in range(num_rooms)]

    def schedule_subject(self, subject, room, day, section):
        self.schedules[room][day][section] = subject

    def unschedule_subject(self, room, day, section):
        self.schedules[room][day][section] = None

    def is_scheduled(self, subject):
        for room in range(self.num_rooms):
            for day in range(self.num_days):
                for section in range(self.num_sections):
                    if self.schedules[room][day][section] == subject:
                        return True
        return False

    def is_feasible(self):
        # Check that each subject is scheduled exactly once
        for subject in range(self.num_subjects):
            if not self.is_scheduled(subject):
                return False

        # Check that the number of students in each room does not exceed its capacity
        for room in range(self.num_rooms):
            num_students = 0
            for day in range(self.num_days):
                for section in range(self.num_sections):
                    subject = self.schedules[room][day][section]
                    if subject is not None:
                        num_students += self.subject_members[subject]
            if num_students > self.room_capacities[room]:
                return False

        # Check that subjects that cannot be scheduled at the same time are not scheduled at the same time
        for subject1, subject2 in self.subject_pairs:
            for day in range(self.num_days):
                for section in range(self.num_sections):
                    if (self.schedules[room][day][section] == subject1 and
                        self.schedules[room][day][section] == subject2):
                        return False

        return True



In [ ]:
def find_empty_slot(schedule):
    for room in range(schedule.num_rooms):
        for day in range(schedule.num_days):
            for section in range(schedule.num_sections):
                if schedule.schedules[room][day][section] is None:
                    return room, day, section
    return None

def move(schedule, subject, new_room, new_day, new_section):
    for room in range(schedule.num_rooms):
        for day in range(schedule.num_days):
            for section in range(schedule.num_sections):
                if schedule.schedules[room][day][section] == subject:
                    schedule.unschedule_subject(room, day, section)
                    break
    schedule.schedule_subject(subject, new_room, new_day, new_section)

def swap(schedule, subject1, subject2):
    room1, day1, section1 = None, None, None
    room2, day2, section2 = None, None, None
    for room in range(schedule.num_rooms):
        for day in range(schedule.num_days):
            for section in range(schedule.num_sections):
                if schedule.schedules[room][day][section] == subject1:
                    room1, day1, section1 = room, day, section
                elif schedule.schedules[room][day][section] == subject2:
                    room2, day2, section2 = room, day, section
    if room1 is not None and room2 is not None:
        schedule.unschedule_subject(room1, day1, section1)
        schedule.unschedule_subject(room2, day2, section2)
        schedule.schedule_subject(subject1, room2, day2, section2)
        schedule.schedule_subject(subject2, room1, day1, section1)

In [2]:
import random
import copy

def find_neighbor(schedule, tabu_list):
    # Make a copy of the current schedule so we can modify it
    neighbor = Schedule(schedule.num_subjects, schedule.num_rooms, schedule.num_days, schedule.num_sections,
                        schedule.subject_members, schedule.room_capacities, schedule.subject_pairs)
    neighbor.schedules = copy.deepcopy(schedule.schedules)

    # Randomly choose a subject to move or swap
    subject1 = random.randint(0, neighbor.num_subjects - 1)
    subject2 = random.randint(0, neighbor.num_subjects - 1)

    # Check if subject1 is scheduled, and if so, find its current time slot
    room1, day1, section1 = None, None, None
    for r in range(neighbor.num_rooms):
        for d in range(neighbor.num_days):
            for s in range(neighbor.num_sections):
                if neighbor.schedules[r][d][s] == subject1:
                    room1, day1, section1 = r, d, s
                    break

    # Check if subject2 is scheduled, and if so, find its current time slot
    room2, day2, section2 = None, None, None
    for r in range(neighbor.num_rooms):
        for d in range(neighbor.num_days):
            for s in range(neighbor.num_sections):
                if neighbor.schedules[r][d][s] == subject2:
                    room2, day2, section2 = r, d, s
                    break

    # If subject1 is not scheduled, try to add it to an empty time slot
    if room1 is None:
        empty_slot = find_empty_slot(neighbor)
        if empty_slot is not None:
            room1, day1, section1 = empty_slot
            move(neighbor, subject1, room1, day1, section1)

    # If subject2 is not scheduled, try to add it to an empty time slot
    if room2 is None:
        empty_slot = find_empty_slot(neighbor)
        if empty_slot is not None:
            room2, day2, section2 = empty_slot
            move(neighbor, subject2, room2, day2, section2)

    # If both subjects are scheduled, try to swap their time slots
    if room1 is not None and room2 is not None:
        # Check if the swap is on the tabu list
        if (subject1, subject2) in tabu_list or (subject2, subject1) in tabu_list:
            # If the swap is tabu, try to move subject1 to an empty time slot instead
            empty_slot = find_empty_slot(neighbor)
            if empty_slot is not None:
                move(neighbor, subject1, *empty_slot)
        else:
            # If the swap is not tabu, go ahead
            swap(neighbor, subject1, subject2)
        
    # Return the neighbor
    return neighbor

